DEseq2 Tutorial Jason Tsai 11th November 2021

# Initiate libraries

In [1]:
library("tidyverse")
library("DESeq2")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’

# Data import
To demonstate the use of DESeqDataSetFromMatrix, we will read in count data from the pasilla package.
We read in a count matrix, which we will name `cts`, and the sample information table, which we will name `coldata`.

In [2]:
# Load the counts matrix
cts <- read.csv(file = "data/PDAC_MICRODISSEC_NAIF/rna_seq.csv",
                row.names = 1)

# Load the metadata
coldata <- read.csv("data/PDAC_MICRODISSEC_NAIF/rna_seq_metadata.csv",
                    row.names = 1)

# load the microdissection densities table
densities <- read.csv(file = "data/MALDI_IHC/microdissection_densities.csv",
                      row.names = 1)

In [3]:
# Exclude the samples that have no microdissection densities
cts <- cts[, colnames(cts) %in% rownames(densities)]
coldata <- coldata[rownames(densities), ]

# Reorder the columns of the counts matrix
cts <- cts[, rownames(densities)]

# Normalize the counts using upper quartile normalization

In [4]:
head(cts)

,BPDAC_023_19_L1_S92,BPDAC_023_26_L1_S5,BPDAC_023_26_L2_S13,X0823_012,X0823_013,BPDAC_025_15_L1_S85,BPDAC_029_26_L1_S70,BPDAC_029_26_L2_S78,X0923_009,X0923_010,⋯,BPDAC_017_29_L1_S67,X0923_023,BPDAC_018_19_L1_S83,BPDAC_018_22_L1_S91,BPDAC_018_22_L2_S4,BPDAC_018_22_L3_S12,BPDAC_022_25_L1_S76,BPDAC_022_25_L2_S84,X0923_026,X0923_027
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSG00000160072,459,572,0,365,312,212,77,86,86,67,⋯,15,171,372,647,354,593,191,4,95,56
ENSG00000234396,0,0,0,5,0,1,0,0,0,13,⋯,0,0,0,6,4,0,0,0,4,0
ENSG00000225972,3364,3967,0,2501,4641,16,54,22,72,41,⋯,21,57,15,5,40,14,58,0,54,25
ENSG00000224315,30,0,0,25,40,27,0,0,85,0,⋯,0,10,16,24,64,4,0,0,13,6
ENSG00000198744,314,167,0,329,330,44,23,44,160,128,⋯,23,201,34,39,58,60,206,1,276,101
ENSG00000279928,1,0,0,0,0,1,0,0,0,0,⋯,0,2,8,6,23,3,2,3,0,0


In [5]:
# Define the function to normalize the counts
uqnorm <- function(rawcounts)
{
  log2(1 + (t(t(rawcounts)/apply(rawcounts, 2, function(x) {
    quantile(x[which(x > 0)], probs = 0.75)})) * 1000))
}

# Normalize the counts
cts_normalized_uqnorm <- uqnorm(cts)

head(cts_normalized_uqnorm)

,BPDAC_023_19_L1_S92,BPDAC_023_26_L1_S5,BPDAC_023_26_L2_S13,X0823_012,X0823_013,BPDAC_025_15_L1_S85,BPDAC_029_26_L1_S70,BPDAC_029_26_L2_S78,X0923_009,X0923_010,⋯,BPDAC_017_29_L1_S67,X0923_023,BPDAC_018_19_L1_S83,BPDAC_018_22_L1_S91,BPDAC_018_22_L2_S4,BPDAC_018_22_L3_S12,BPDAC_022_25_L1_S76,BPDAC_022_25_L2_S84,X0923_026,X0923_027
ENSG00000160072,10.884086,10.931638,0,10.931381,10.246750,10.148082,9.562055,9.499226,9.137226,9.541957,⋯,6.398031,10.245275,11.698022,11.860851,11.205182,10.719097,9.256707,8.385143,9.450241,9.546251
ENSG00000234396,0.000000,0.000000,0,4.793790,0.000000,2.666210,0.000000,0.000000,0.000000,7.184324,⋯,0.000000,0.000000,0.000000,5.149046,4.790077,0.000000,0.000000,0.000000,4.926558,0.000000
ENSG00000225972,13.757038,13.724967,0,13.707287,14.140460,6.435654,9.050969,7.538181,8.881384,8.834647,⋯,6.878562,8.662687,7.076052,4.894121,8.064294,5.349533,7.542658,0.000000,8.636838,8.385143
ENSG00000224315,6.959501,0.000000,0,7.073497,7.291327,7.183735,0.000000,0.000000,9.120382,0.000000,⋯,0.000000,6.168361,7.168493,7.118225,8.740343,3.628031,0.000000,0.000000,6.593776,6.339850
ENSG00000198744,10.336709,9.157261,0,10.781653,10.327605,7.884440,7.823305,8.534295,10.031703,10.474945,⋯,7.008740,10.478296,8.250637,7.814665,8.598673,7.421682,9.365607,6.398031,10.987557,10.396248
ENSG00000279928,2.354798,0.000000,0,0.000000,0.000000,2.666210,0.000000,0.000000,0.000000,0.000000,⋯,0.000000,3.924518,6.178487,5.149046,7.269906,3.251375,2.886528,7.971544,0.000000,0.000000


## Verify the integrity of the data

In [6]:
head(coldata)

,patient_ID2,sample_ID,ID_DNA,N..histo,ID_Anapath,Idpatient_bloc,Idpatient,sample_ID_4_merge,path_svs,ID_scan,⋯,DECES.1.OUI,date_décès,OS..jours.,OS..360.jours.,OS..mois.,PFS..jours.,PFS.formule,PFS..mois.,Unnamed..27,infiltrat_lymphocytaire
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
BPDAC_023_19_L1_S92,BPDAC_023,BPDAC_023_19_L1,,B_12AG01290_19_L1,12AG01290_19_L1,12AG01290_19,12AG01290,12AG01290-19,\\bob\histology\datasets\PDAC_Bjn_100\PDAC_MICRODISSEC_NAIF\old_names\12AG01290-19_MDNF01_HES.svs,12AG01290-19_MDNF01_HES.svs,⋯,0,,356,351,11.50820,316,321,10.36066,,fort
BPDAC_023_26_L1_S5,BPDAC_023,BPDAC_023_26_L1_MDNF01,,B_12AG01290_26_L1,12AG01290_26_L1,12AG01290_26,12AG01290,12AG01290-26,\\bob\histology\datasets\PDAC_Bjn_100\PDAC_MICRODISSEC_NAIF\old_names\12AG01290-26_MDNF01_HES.svs,12AG01290-26_MDNF01_HES.svs,⋯,0,,356,351,11.50820,316,321,10.36066,,modere
BPDAC_023_26_L2_S13,BPDAC_023,BPDAC_023_26_L2_MDNF01,,B_12AG01290_26_L2,12AG01290_26_L2,12AG01290_26,12AG01290,12AG01290-26,\\bob\histology\datasets\PDAC_Bjn_100\PDAC_MICRODISSEC_NAIF\old_names\12AG01290-26_MDNF01_HES.svs,12AG01290-26_MDNF01_HES.svs,⋯,0,,356,351,11.50820,316,321,10.36066,,faible
X0823_012,BPDAC_023,BPDAC_023_26_L1_MDNF02,,B_12AG01290_26_L1,12AG01290_26_L1,12AG01290_26,12AG01290,12AG01290-26,\\bob\histology\datasets\PDAC_Bjn_100\PDAC_MICRODISSEC_NAIF\old_names\12AG01290-26_MDNF02_HES.svs,12AG01290-26_MDNF02_HES.svs,⋯,0,,356,351,11.50820,316,321,10.36066,,faible
X0823_013,BPDAC_023,BPDAC_023_26_L2_MDNF02,,B_12AG01290_26_L2,12AG01290_26_L2,12AG01290_26,12AG01290,12AG01290-26,\\bob\histology\datasets\PDAC_Bjn_100\PDAC_MICRODISSEC_NAIF\old_names\12AG01290-26_MDNF02_HES.svs,12AG01290-26_MDNF02_HES.svs,⋯,0,,356,351,11.50820,316,321,10.36066,,modere
BPDAC_025_15_L1_S85,BPDAC_025,BPDAC_025_15_L1,,B_12AG01824_15_L1,12AG01824_15_L1,12AG01824_15,12AG01824,12AG01824-15,\\bob\histology\datasets\PDAC_Bjn_100\PDAC_MICRODISSEC_NAIF\old_names\12AG01824-15_MDNF01_HES.svs,12AG01824-15_MDNF01_HES.svs,⋯,0,,1297,1279,41.93443,1279,1297,41.93443,,modere


In [7]:
# Check again if the names are the same regardless of the order
all(rownames(coldata) %in% colnames(cts))

[1] TRUE

In [8]:
# Check again if order is the same
all(rownames(coldata) == colnames(cts))

[1] TRUE

# Save the normalized counts

In [9]:
# Save the normalized counts
cts_normalized_uqnorm |>
  write.csv(file = "data/PDAC_MICRODISSEC_NAIF/rna_seq_selected_uqnorm.csv")

# Construct a DESeqDataSet
Now both the coldata and `cts` have the same row names in the same order.
With the count matrix, `cts`, and the sample information, `coldata`, we can construct a `DESeqDataSet`:

In [10]:
# Make a factor of random positive and negative samples
coldata$condition <- factor(sample(c("Positive", "Negative"), size = nrow(coldata), replace = TRUE))

dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~ condition)
dds

class: DESeqDataSet 
dim: 61806 87 
metadata(1): version
assays(1): counts
rownames(61806): ENSG00000160072 ENSG00000234396 ... ENSG00000210195
  ENSG00000210196
rowData names(0):
colnames(87): BPDAC_023_19_L1_S92 BPDAC_023_26_L1_S5 ... X0923_026
  X0923_027
colData names(94): patient_ID2 sample_ID ... infiltrat_lymphocytaire
  condition

## Extracting transformed values
These transformation functions return an object of class DESeqTransform which is a subclass of RangedSummarizedExperiment.
For ~20 samples, running on a newly created DESeqDataSet. The assay function is used to extract the matrix of normalized values.

In [11]:
vsd <- dds |> vst()
vsd

class: DESeqTransform 
dim: 61806 87 
metadata(1): version
assays(1): ''
rownames(61806): ENSG00000160072 ENSG00000234396 ... ENSG00000210195
  ENSG00000210196
rowData names(4): baseMean baseVar allZero dispFit
colnames(87): BPDAC_023_19_L1_S92 BPDAC_023_26_L1_S5 ... X0923_026
  X0923_027
colData names(95): patient_ID2 sample_ID ... condition sizeFactor

In [12]:
vsd |> assay() |> head()

,BPDAC_023_19_L1_S92,BPDAC_023_26_L1_S5,BPDAC_023_26_L2_S13,X0823_012,X0823_013,BPDAC_025_15_L1_S85,BPDAC_029_26_L1_S70,BPDAC_029_26_L2_S78,X0923_009,X0923_010,⋯,BPDAC_017_29_L1_S67,X0923_023,BPDAC_018_19_L1_S83,BPDAC_018_22_L1_S91,BPDAC_018_22_L2_S4,BPDAC_018_22_L3_S12,BPDAC_022_25_L1_S76,BPDAC_022_25_L2_S84,X0923_026,X0923_027
ENSG00000160072,8.145062,8.341551,5.116169,8.342406,7.882671,7.787172,8.757379,7.487186,7.358658,7.511037,⋯,6.268452,7.858504,8.537024,8.688237,8.111345,7.863419,7.596872,8.171762,7.403251,7.620162
ENSG00000234396,5.116169,5.116169,5.116169,5.575576,5.116169,5.326773,5.116169,5.116169,5.116169,6.265784,⋯,5.116169,5.116169,5.116169,5.553333,5.493813,5.116169,5.116169,5.116169,5.633403,5.116169
ENSG00000225972,10.702245,10.863453,5.116169,10.847690,11.357258,5.947771,8.332811,6.411068,7.196060,7.061471,⋯,6.466289,6.843739,5.963481,5.515496,6.281830,5.602711,6.592640,5.116169,6.909162,6.892850
ENSG00000224315,6.025646,5.116169,5.116169,6.126977,6.237218,6.186736,5.116169,5.116169,7.347691,5.116169,⋯,5.116169,5.875005,5.990461,5.980846,6.568903,5.377115,5.116169,5.116169,6.037872,6.027392
ENSG00000198744,7.733498,7.088121,5.116169,8.222636,7.943229,6.464757,7.424035,6.894736,7.991938,8.198857,⋯,6.524640,8.034164,6.370379,6.208698,6.504152,6.108508,7.673535,6.841333,8.558770,8.258313
ENSG00000279928,5.284883,5.116169,5.116169,5.116169,5.116169,5.326773,5.116169,5.116169,5.116169,5.116169,⋯,5.116169,5.458651,5.739041,5.553333,6.009953,5.342232,5.401998,7.855183,5.116169,5.116169


In [13]:
# Save the variance stabilized data
vsd |>
  assay() |>
  as.data.frame() |>
  write.csv(file = "data/PDAC_MICRODISSEC_NAIF/rna_seq_selected_vst.csv")